# Lesson 14. SimPy &mdash; multiple arrival types, user-defined distributions

### SA421 Fall 2015

## Warm up

**Problem.** Passengers arrive at Bellman Air's check-in counter in the following proportions:

| Passenger Type   | Probability |
| ---------------- | ----------- |
| Economy (0)      | 0.60        |
| Business (1)     | 0.30        |
| First (2)        | 0.10        |


Write a function called `passengerType` that generates random variates 0, 1 or 2 from this distribution: let 0 = economy class, 1 = business class, and 2 = first class. 

*Hint.* Use the inverse transform method for discrete distributions.

In [ ]:
# Import infinity from NumPy
from numpy import inf

# Import NumPy's random number generator
from numpy.random import rand

# Import bisect_right from bisect
from bisect import bisect_left

# Passenger type distribution: 
# Economy = 0, Business = 1, First = 2
def passengerType():
    ##
    ## This part needs to be filled in...
    ##
    
    # Return generated variate
    return variate

Test your function in the code cell below:

In [ ]:
variates = [passengerType() for i in range(50)]
print("50 randomly generated passenger types = {0}".format(variates))

## Overview

* Today's primary objective: how can we model the arrival of different customer types?


* Secondary objective: using custom (discrete) distributions in Python.

## Probability distribution over different arrival types

**Problem.** Passengers arrive at Bellman Air's check-in counter according to an exponential interarrival-time distribution with mean 1.6 minutes. 60% of the passengers are traveling economy class, 30% business class, and 10% first class. There is a single queue, and 3 agents at the counter. Service times for economy class passengers are uniformly distributed between 4 and 12 minutes; for business class passengers, uniformly distributed between 5 and 10 minutes; for first class passengers, uniformly distributed between 6 and 9 minutes.

Simulate the system for 2 hours.

* Below is code for a simpler version of this problem that assumes that all passengers are traveling economy class.


* The service time parameters for all passenger types have already been defined.


* No monitors are set up and no performance measures are computed.


* The simulation is set up to print information about passenger arrivals.

In [ ]:
##### Setup #####
# Import everything from SimPy
from SimPy.Simulation import *

# Import seed initializer and random sampling functions from NumPy
from numpy.random import seed, exponential, uniform

In [ ]:
##### Parameters #####
class P:
    # Passengers arrive at the entrance with exponentially distributed
    # interarrival times with mean 1.6 minutes
    interarrivalTimeMean = 1.6
    
    # Service times for economy class passengers is 
    # uniformly distributed between 4 and 12 minutes
    serviceTimeEconMin = 4
    serviceTimeEconMax = 12
    
    # Service times for business class passengers is 
    # uniformly distributed between 5 and 10 minutes
    serviceTimeBusMin = 5
    serviceTimeBusMax = 10

    # Service times for first class passengers is 
    # uniformly distributed between 6 and 9 minutes
    serviceTimeFirstMin = 6
    serviceTimeFirstMax = 9
    
    # Number of agents: 3
    nAgents = 3
    
    # Simluate for 2 continuous hours
    simulationTimeMax = 2 * 60
    

##### Processes #####
# Economy class passenger
class PassengerEcon(Process):
    def behavior(self):
        # Passenger arrives, joins check-in counter queue
        print("Time {0}: {1} arrives, joins queue".format(now(), self.name))
        yield request, self, R.counter
        
        # Passenger is released from queue and starts service
        serviceTime = uniform(low = P.serviceTimeEconMin, high = P.serviceTimeEconMax)
        yield hold, self, serviceTime
        
        # Passenger finishes service, leaves
        yield release, self, R.counter

# Entrance
class Entrance(Process):
    def behavior(self):
        # At the start of the simulation, no passengers have arrived
        nPassengers = 0
        
        # Passenger arrivals
        while True:
            # Wait until the next arrival
            interarrivalTime = exponential(scale = P.interarrivalTimeMean)
            yield hold, self, interarrivalTime
            
            # Create a new passenger using the template defined in the Passenger class
            c = PassengerEcon(name="Passenger {0} (Economy)".format(nPassengers))
            
            # Activate the passenger's behavior
            activate(c, c.behavior())

            # Count this new passenger
            nPassengers += 1

            
##### Resources #####
class R:
    # Check-in counter
    counter = None


##### Model #####
def model(inputSeed):
    # Initialize SimPy 
    initialize()

    # Initialize a seed for the random number generator
    seed(inputSeed)

    # Create the counter resource
    R.counter = Resource(capacity = P.nAgents)

    # Activate the entrance
    e = Entrance()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = P.simulationTimeMax)

In [ ]:
model(123)

* Let's incorporate the random passenger type generator we created in the warm up.


* Like with resources, monitors, and streams, by convention, we will put all custom distributions in their own class. For distributions, we'll use `D`.

In [ ]:
##### Distributions #####
class D:


* To generate a random passenger type with the function defined above, we use `D.passengerType()`.

* Next, let's create different processes for each passenger type.


* Based on the process for economy class passengers, let's create a process for business class passengers:

In [ ]:
# Edit PassengerEcon process to create business class passenger process
class PassengerEcon(Process):
    def behavior(self):
        # Passenger arrives, joins check-in counter queue
        print("Time {0}: {1} arrives, joins queue".format(now(), self.name))
        yield request, self, R.counter
        
        # Passenger is released from queue and starts service
        serviceTime = uniform(low = P.serviceTimeEconMin, high = P.serviceTimeEconMax)
        yield hold, self, serviceTime
        
        # Passenger finishes service, leaves
        yield release, self, R.counter

* Similarly, let's create a process for first class passengers:

In [ ]:
# Edit PassengerEcon process to create first class passenger process
class PassengerEcon(Process):
    def behavior(self):
        # Passenger arrives, joins check-in counter queue
        print("Time {0}: {1} arrives, joins queue".format(now(), self.name))
        yield request, self, R.counter
        
        # Passenger is released from queue and starts service
        serviceTime = uniform(low = P.serviceTimeEconMin, high = P.serviceTimeEconMax)
        yield hold, self, serviceTime
        
        # Passenger finishes service, leaves
        yield release, self, R.counter

* With processes defined for each passenger type, we need to then have them arrive at the entrance in the correct proportions.


* To do so, let's redefine the Entrance process:

In [ ]:
# Edit Entrance process so that different passenger types 
#  arrive in the correct proportions
class Entrance(Process):
    def behavior(self):
        # At the start of the simulation, no passengers have arrived
        nPassengers = 0
        
        # Passenger arrivals
        while True:
            # Wait until the next arrival
            interarrivalTime = exponential(scale = P.interarrivalTimeMean)
            yield hold, self, interarrivalTime
            
            # Create a new passenger using the template defined in the Passenger class
            c = PassengerEcon(name="Passenger {0} (Economy)".format(nPassengers))
            
            # Activate the passenger's behavior
            activate(c, c.behavior())

            # Count this new passenger
            nPassengers += 1

* Finally, we're ready to run this simulation.


* Let's run it once to see what happens.

In [ ]:
model(123)

## Multiple arrival streams

**Problem.** Passengers arrive at Bellman Air's check-in counter according to an exponential interarrival-time distribution: <span style="color:#d00000;">the distribution for economy class passengers has mean 3 minutes, business class 5 minutes, first class 15 minutes</span>. There is a single queue, and 3 agents at the counter. Service times for economy class passengers are uniformly distributed between 4 and 12 minutes; for business class passengers, uniformly distributed between 5 and 10 minutes; for first class passengers, uniformly distributed between 6 and 9 minutes.

Simulate the system for 2 hours.

* Below is code for a simpler version of this problem that assumes that all passengers are traveling economy class.


* The interarrival time and service time parameters for all passenger types have already been defined.


* No monitors are set up and no performance measures are computed.


* The simulation is set up to print information about passenger arrivals.

In [ ]:
##### Parameters #####
class P:
    # Economy class passengers arrive at the entrance 
    # with exponentially distributed interarrival times 
    # with mean 3 minutes
    interarrivalTimeEconMean = 3
    
    # Business class passengers arrive at the entrance 
    # with exponentially distributed interarrival times 
    # with mean 5 minutes
    interarrivalTimeBusMean = 5
    
    # First class passengers arrive at the entrance 
    # with exponentially distributed interarrival times 
    # with mean 15 minutes
    interarrivalTimeFirstMean = 15
    
    # Service times for economy class passengers is 
    # uniformly distributed between 4 and 12 minutes
    serviceTimeEconMin = 4
    serviceTimeEconMax = 12
    
    # Service times for business class passengers is 
    # uniformly distributed between 5 and 10 minutes
    serviceTimeBusMin = 5
    serviceTimeBusMax = 10

    # Service times for first class passengers is 
    # uniformly distributed between 6 and 9 minutes
    serviceTimeFirstMin = 6
    serviceTimeFirstMax = 9
    
    # Number of agents: 3
    nAgents = 3
    
    # Simluate for 2 continuous hours
    simulationTimeMax = 2 * 60
    

##### Processes #####
# Economy class passenger
class PassengerEcon(Process):
    def behavior(self):
        # Passenger arrives, joins check-in counter queue
        print("Time {0}: {1} arrives, joins queue".format(now(), self.name))
        yield request, self, R.counter
        
        # Passenger is released from queue and starts service
        serviceTime = uniform(low = P.serviceTimeEconMin, high = P.serviceTimeEconMax)
        yield hold, self, serviceTime
        
        # Passenger finishes service, leaves
        yield release, self, R.counter

# Economy class entrance
class EntranceEcon(Process):
    def behavior(self):
        # At the start of the simulation, no passengers have arrived
        nPassengers = 0
        
        # Passenger arrivals
        while True:
            # Wait until the next arrival
            interarrivalTime = exponential(scale = P.interarrivalTimeEconMean)
            yield hold, self, interarrivalTime
            
            # Create a new economy class passenger
            c = PassengerEcon(name="Passenger {0} (Economy)".format(nPassengers))
            
            # Activate the passenger's behavior
            activate(c, c.behavior())

            # Count this new passenger
            nPassengers += 1

            
##### Resources #####
class R:
    # Check-in counter
    counter = None


##### Model #####
def model(inputSeed):
    # Initialize SimPy 
    initialize()

    # Initialize a seed for the random number generator
    seed(inputSeed)

    # Create the counter resource
    R.counter = Resource(capacity = P.nAgents)

    # Activate the economy passenger entrance
    e = EntranceEcon()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = P.simulationTimeMax)

* Again, we need create different processes that defines the beavhior for each passenger type.


* Let's create a process to define the behavior of business class passengers:

In [ ]:
# Business class passenger


* In addition, let's create a process to define the behavior of first class passengers:

In [ ]:
# First class passenger


* To have the three types of passengers arrive simultaneously, we can create a <span style="color:#d00000;">separate entrance for each type of passenger</span> &mdash; i.e., multiple arrival streams.


* Note that these separate entrances don't necessarily have to represent physically different entrances.


* We already have an entrance for the economy class passengers.


* Based on the entrance for the economy class passengers, let's create an entrance for the business class passengers:

In [ ]:
# Edit EntranceEcon process to create an entrance for business class passengers
class EntranceEcon(Process):
    def behavior(self):
        # At the start of the simulation, no passengers have arrived
        nPassengers = 0
        
        # Passenger arrivals
        while True:
            # Wait until the next arrival
            interarrivalTime = exponential(scale = P.interarrivalTimeEconMean)
            yield hold, self, interarrivalTime
            
            # Create a new economy class passenger
            c = PassengerEcon(name="Passenger {0} (Economy)".format(nPassengers))
            
            # Activate the passenger's behavior
            activate(c, c.behavior())

            # Count this new passenger
            nPassengers += 1

* Let's also create an entrance for the first class passengers:

In [ ]:
# Edit EntranceEcon process to create an entrance for first class passengers
class EntranceEcon(Process):
    def behavior(self):
        # At the start of the simulation, no passengers have arrived
        nPassengers = 0
        
        # Passenger arrivals
        while True:
            # Wait until the next arrival
            interarrivalTime = exponential(scale = P.interarrivalTimeEconMean)
            yield hold, self, interarrivalTime
            
            # Create a new economy class passenger
            c = PassengerEcon(name="Passenger {0} (Economy)".format(nPassengers))
            
            # Activate the passenger's behavior
            activate(c, c.behavior())

            # Count this new passenger
            nPassengers += 1

* Finally, before running the simulation, we need to activate all three entrances in `model()`:

In [ ]:
# Redefine model() to activate all three entrances
def model(inputSeed):
    # Initialize SimPy 
    initialize()

    # Initialize a seed for the random number generator
    seed(inputSeed)

    # Create the counter resource
    R.counter = Resource(capacity = P.nAgents)

    # Activate the economy passenger entrance
    e = EntranceEcon()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = P.simulationTimeMax)

* Let's run this simulation once to see what happens.

In [ ]:
model(123)

## If we have time... with a neighbor

**Problem.** Below is the SimPy code for the Bellman Air example above with multiple arrival streams, combined into one cell.

Bellman Air is considering having two separate counters: one for economy class passengers, and one for business and first class passengers. The economy class counter would have 2 agents, and the business/first class counter would have 1 agent. Modify the SimPy code below to model this proposed change.

<span style="color:#004276;">In order to model this change, we need to introduce another resource, so that we can model two separate counters. Then, we need to make sure each passenger type requests the correct resource.</span>

In [ ]:
##### Parameters #####
class P:
    # Economy class passengers arrive at the entrance 
    # with exponentially distributed interarrival times 
    # with mean 3 minutes
    interarrivalTimeEconMean = 3
    
    # Business class passengers arrive at the entrance 
    # with exponentially distributed interarrival times 
    # with mean 5 minutes
    interarrivalTimeBusMean = 5
    
    # First class passengers arrive at the entrance 
    # with exponentially distributed interarrival times 
    # with mean 15 minutes
    interarrivalTimeFirstMean = 15
    
    # Service times for economy class passengers is 
    # uniformly distributed between 4 and 12 minutes
    serviceTimeEconMin = 4
    serviceTimeEconMax = 12
    
    # Service times for business class passengers is 
    # uniformly distributed between 5 and 10 minutes
    serviceTimeBusMin = 5
    serviceTimeBusMax = 10

    # Service times for first class passengers is 
    # uniformly distributed between 6 and 9 minutes
    serviceTimeFirstMin = 6
    serviceTimeFirstMax = 9
    
    # Number of agents: 3
    nAgents = 3

    # Simluate for 2 continuous hours
    simulationTimeMax = 2 * 60
    

##### Processes #####
# Economy class passenger
class PassengerEcon(Process):
    def behavior(self):
        # Passenger arrives, joins check-in counter queue
        print("Time {0}: {1} arrives, joins queue".format(now(), self.name))
        yield request, self, R.counter
        
        # Passenger is released from queue and starts service
        serviceTime = uniform(low = P.serviceTimeEconMin, high = P.serviceTimeEconMax)
        yield hold, self, serviceTime
        
        # Passenger finishes service, leaves
        yield release, self, R.counter

# Business class passenger
class PassengerBus(Process):
    def behavior(self):
        # Passenger arrives, joins check-in counter queue
        print("Time {0}: {1} arrives, joins queue".format(now(), self.name))
        yield request, self, R.counter
        
        # Passenger is released from queue and starts service
        serviceTime = uniform(low = P.serviceTimeBusMin, high = P.serviceTimeBusMax)
        yield hold, self, serviceTime
        
        # Passenger finishes service, leaves
        yield release, self, R.counter 
        
# First class passenger
class PassengerFirst(Process):
    def behavior(self):
        # Passenger arrives, joins check-in counter queue
        print("Time {0}: {1} arrives, joins queue".format(now(), self.name))
        yield request, self, R.counter
        
        # Passenger is released from queue and starts service
        serviceTime = uniform(low = P.serviceTimeFirstMin, high = P.serviceTimeFirstMax)
        yield hold, self, serviceTime
        
        # Passenger finishes service, leaves
        yield release, self, R.counter
        
# Economy class entrance
class EntranceEcon(Process):
    def behavior(self):
        # At the start of the simulation, no passengers have arrived
        nPassengers = 0
        
        # Passenger arrivals
        while True:
            # Wait until the next arrival
            interarrivalTime = exponential(scale = P.interarrivalTimeEconMean)
            yield hold, self, interarrivalTime
            
            # Create a new economy class passenger
            c = PassengerEcon(name="Passenger {0} (Economy)".format(nPassengers))
            
            # Activate the passenger's behavior
            activate(c, c.behavior())

            # Count this new passenger
            nPassengers += 1

# Business class entrance
class EntranceBus(Process):
    def behavior(self):
        # At the start of the simulation, no passengers have arrived
        nPassengers = 0
        
        # Passenger arrivals
        while True:
            # Wait until the next arrival
            interarrivalTime = exponential(scale = P.interarrivalTimeBusMean)
            yield hold, self, interarrivalTime
            
            # Create a new business class passenger
            c = PassengerBus(name="Passenger {0} (Business)".format(nPassengers))
            
            # Activate the passenger's behavior
            activate(c, c.behavior())

            # Count this new passenger
            nPassengers += 1

# First class entrance
class EntranceFirst(Process):
    def behavior(self):
        # At the start of the simulation, no passengers have arrived
        nPassengers = 0
        
        # Passenger arrivals
        while True:
            # Wait until the next arrival
            interarrivalTime = exponential(scale = P.interarrivalTimeFirstMean)
            yield hold, self, interarrivalTime
            
            # Create a new first class passenger
            c = PassengerFirst(name="Passenger {0} (First)".format(nPassengers))
            
            # Activate the passenger's behavior
            activate(c, c.behavior())

            # Count this new passenger
            nPassengers += 1


##### Resources #####
class R:
    # Check-in counter
    counter = None


##### Model #####
def model(inputSeed):
    # Initialize SimPy 
    initialize()

    # Initialize a seed for the random number generator
    seed(inputSeed)

    # Create the counter resource
    R.counter = Resource(capacity = P.nAgents)

    # Activate the economy class passenger entrance
    ee = EntranceEcon()
    activate(ee, ee.behavior())
    
    # Activate the business class passenger entrance
    eb = EntranceBus()
    activate(eb, eb.behavior())
    
    # Activate the business class passenger entrance
    ef = EntranceFirst()
    activate(ef, ef.behavior())
    
    # Run the simulation
    simulate(until = P.simulationTimeMax)

In [ ]:
model(123)